# Book Recommendation System - Part 1 (EDA)

## Import libraries

In [28]:
# Import libraries
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from scipy import stats

## Import Dataset

In this project, we will use Goodreads dataset. The dataset was originally scraped from the Goodreads API in September 2017 by Zygmunt Zając and updated by Olivier Simard-Hanley. This dataset consist of 5 files which are:
- ratings.csv: contains user ratings for books they read
- books_enriched.csv: contains metadata for each book (book ID, title, authors, year published, etc)
- to-read.csv: contains books marked "to read" by users
- book_tag.csv: contains tags/shelves/genres assigned by users to books
- tag.csv: contains the tag names corresponding to the tag ids in book_tag.csv

I will only use two files which are user ratings and metadata of the books.

In [29]:
pd.options.display.float_format = '{:.2f}'.format
r = pd.read_csv('Dataset/ratings.csv')
b = pd.read_csv('Dataset/books_enriched.csv')

## Data Understanding

### 1. Rating

In [30]:
r.head()

,user_id,book_id,rating
0,1,258,5
1,2,4081,4
2,2,260,5
3,2,9296,5
4,2,2318,3


In **dataset r**, there are three columns, which are:
1. `user_id`: user identification number
2. `book_id`: book identification number
3. `rating`: rating given by `user_id`

In [31]:
r.shape

(5976479, 3)

In [32]:
for col in r.columns:
    print(f"Number of {col} is {r[col].nunique()}")

Number of user_id is 53424
Number of book_id is 10000
Number of rating is 5


There are 5,976,479 ratings given by 53,424 people on 10,000 books.

### 2. Books

Now let's explore our book metadata, we can first chek the shape of the data.

In [33]:
b.shape

(10000, 30)

It has 10000 rows and 30 columns. We want to display values in all columns, therefore we will transpose the table and only show first three rows.

In [34]:
b.head()

,Unnamed: 0,index,authors,average_rating,best_book_id,book_id,books_count,description,genres,goodreads_book_id,...,ratings_3,ratings_4,ratings_5,ratings_count,small_image_url,title,work_id,work_ratings_count,work_text_reviews_count,authors_2
0,0,0,['Suzanne Collins'],4.34,2767052,1,272,WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,"['young-adult', 'fiction', 'fantasy', 'science...",2767052,...,560092,1481305,2706317,4780653,https://images.gr-assets.com/books/1447303603s...,"The Hunger Games (The Hunger Games, #1)",2792775,4942365,155254,['Suzanne Collins']
1,1,1,"['J.K. Rowling', 'Mary GrandPré']",4.44,3,2,491,Harry Potter's life is miserable. His parents ...,"['fantasy', 'fiction', 'young-adult', 'classics']",3,...,455024,1156318,3011543,4602479,https://images.gr-assets.com/books/1474154022s...,Harry Potter and the Sorcerer's Stone (Harry P...,4640799,4800065,75867,"['J.K. Rowling', 'Mary GrandPré']"
2,2,2,['Stephenie Meyer'],3.57,41865,3,226,About three things I was absolutely positive.\...,"['young-adult', 'fantasy', 'romance', 'fiction...",41865,...,793319,875073,1355439,3866839,https://images.gr-assets.com/books/1361039443s...,"Twilight (Twilight, #1)",3212258,3916824,95009,['Stephenie Meyer']
3,3,3,['Harper Lee'],4.25,2657,4,487,The unforgettable novel of a childhood in a sl...,"['classics', 'fiction', 'historical-fiction', ...",2657,...,446835,1001952,1714267,3198671,https://images.gr-assets.com/books/1361975680s...,To Kill a Mockingbird,3275794,3340896,72586,['Harper Lee']
4,4,4,['F. Scott Fitzgerald'],3.89,4671,5,1356,Alternate Cover Edition ISBN: 0743273567 (ISBN...,"['classics', 'fiction', 'historical-fiction', ...",4671,...,606158,936012,947718,2683664,https://images.gr-assets.com/books/1490528560s...,The Great Gatsby,245494,2773745,51992,['F. Scott Fitzgerald']


In this dataset, there are 30 columns. However, after investigating them, there are several columns that are repeated. For example too many book_id columns, 2 title columns, 2 authors column, details of the number of reviews per rating for each book, etc. In order to make cleaner metadata, I will do data preprocessing.

**Columns in dataset b**:<br>
- Identification number related (`book_id`, `goodreads_book_id`, `best_book_id`, `work_id`, `isbn`, `isbn13`)
- Title related (`original_title`, `title`)
- Authors related (`authors`, `authors_2`)
- Publication year related (`original_publication_year`, `publishDate`: the publication date)
- Rating related (`average_rating`, `ratings_count`: number of review, `work_ratings_count`, `work_text_reviews_count`, `ratings_1`, `ratings_2`, `ratings_3`, `ratings_4`, `ratings_5`)
- Image Url (`image_url`, `small_image_url`)
- `books_count`: number of edition available
- `language_code`: abbreviated language tags for all books
- `genres`: the genre tags taken from the top shelves users have assigned to a book. Only the main Goodreads genres have been retained
- `pages`: the total page count
- `description`: a free text summarizing the book's content
- Others (`Unnamed: 0`, `index`)

## Statistical Summary

Before we decide which columns we will choose, let's explore the columns first. To make it easier, we can divide the columns to numerical columns and categorical columns and then check the statistics of our data.

In [35]:
#divide the columns to numerical and categorical value
nums = b.select_dtypes(include=[np.number]).columns
cats = b.select_dtypes(exclude=[np.number]).columns

In [36]:
b[nums].describe().T

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,10000.00,4999.50,2886.90,0.00,2499.75,4999.50,7499.25,9999.00
index,10000.00,4526.32,2567.92,0.00,2351.75,4567.50,6661.25,9999.00
average_rating,10000.00,4.00,0.25,2.47,3.85,4.02,4.18,4.82
best_book_id,10000.00,5471213.58,7827329.89,1.00,47911.75,425123.50,9636112.50,35534230.00
book_id,10000.00,5000.50,2886.90,1.00,2500.75,5000.50,7500.25,10000.00
books_count,10000.00,75.71,170.47,1.00,23.00,40.00,67.00,3455.00
goodreads_book_id,10000.00,5264696.51,7575461.86,1.00,46275.75,394965.50,9382225.25,33288638.00
isbn13,9415.00,9755044298883.46,442861920665.57,195170342.00,9780316192995.00,9780451528640.00,9780830777175.00,9790007672390.00
original_publication_year,9979.00,1981.99,152.58,-1750.00,1990.00,2004.00,2011.00,2017.00
pages,9927.00,359.42,216.95,0.00,250.00,336.00,424.00,5216.00


- There are many unique columns: `Unnamed: 0`, `index`, `best_book_id`, `book_id`, `goodreads_book_id`, `isbn13`, `work_id`. The column `book_id` will be used to merge to rating table, so the other columns will be dropped.

- The oldest book was written in -1750 which means 1750 BC.
- Mean of `average_rating` is 4.002 (pretty good!)
- We found weird number of pages (0), we found that book should have at least 80 pages to be published
- The most popular book has 4,780,653 reviews, the least popular has 2,716 reviews.

In [37]:
b[cats].describe().T

,count,unique,top,freq
authors,10000,5057,['Stephen King'],58
description,9943,9772,The first in a spectacularly genre-mashing adv...,4
genres,10000,4351,"['fantasy', 'fiction']",152
image_url,10000,6669,https://s.gr-assets.com/assets/nophoto/book/11...,3332
isbn,9300,9300,439023483,1
language_code,10000,41,eng,9680
original_title,9415,9274,,5
publishDate,9992,4679,"(None, None, None)",75
small_image_url,10000,6669,https://s.gr-assets.com/assets/nophoto/book/50...,3332
title,10000,9964,Selected Poems,4


- The most popular author is Stephen King
- Selected Poems appears 4 times. Is it the same book?
- Because we already has `original_publication_year`, `publishDate` will be dropped
- `authors`, `genres`, `publishDate`, and `authors_2` has unnecessary characters

## Data Preprocessing

Our next step is data preprocessing. In building machine learning models, we should check for missing values and duplicated rows in our data. In this process we should also transform data into a format that is easier and more effective to process, so that our model will produce more accurate results.

### Missing Value and Duplicated Rows

In [38]:
#check missing value
dataset = [r, b]
for data in dataset:
    print(data.isnull().values.any())

False
True


In [39]:
#check duplicated rows
for data in dataset:
    print(data.duplicated().values.any())

False
False


In [40]:
b.isnull().sum()

Unnamed: 0                     0
index                          0
authors                        0
average_rating                 0
best_book_id                   0
book_id                        0
books_count                    0
description                   57
genres                         0
goodreads_book_id              0
image_url                      0
isbn                         700
isbn13                       585
language_code                  0
original_publication_year     21
original_title               585
pages                         73
publishDate                    8
ratings_1                      0
ratings_2                      0
ratings_3                      0
ratings_4                      0
ratings_5                      0
ratings_count                  0
small_image_url                0
title                          0
work_id                        0
work_ratings_count             0
work_text_reviews_count        0
authors_2                      0
dtype: int

In [41]:
#make a copy 
books = b.copy()

In [42]:
percent_missing = books.isnull().sum() * 100 / len(books)
missing_value = (pd.DataFrame({'percent_missing': percent_missing})).sort_values(by=['percent_missing'], ascending=False)
missing_value[missing_value['percent_missing']>0]

,percent_missing
isbn,7.00
original_title,5.85
isbn13,5.85
pages,0.73
description,0.57
original_publication_year,0.21
publishDate,0.08


- Impute `original_publication_year` by using `publishDate`, then drop `publishDate`. I chose `original_publication_year` because it has the same format meanwhile `publishDate` has various data styles.
- Impute `pages` with median.
- Impute `description` with book's title
- Drop `isbn`, `original_title`, `isbn13`, there is no need to impute these columns.

In [43]:
#compare original_publication_year and publishdate when original_publication_year is null
books[books['original_publication_year'].isnull()][['original_publication_year', 'publishDate']]

,original_publication_year,publishDate
198,NaN,October 28th 2008
3249,NaN,February 7th 2006
3881,NaN,September 29th 2009
4252,NaN,November 10th 2010
4303,NaN,June 23rd 2009
4392,NaN,April 8th 2013
5001,NaN,November 9th 2004
5208,NaN,December 6th 2010
5648,NaN,October 11th 2006
7163,NaN,November 25th 2004


Lucky that all rows that are invalid in `original_publication_year` has the year in `publishDate`. Therefore, we can get year in `publishDate` and impute it to `original_publication_year`.

In [44]:
#get year from publishDate to original_publication_year
books['publishDate'].replace(r"[(')]", "", regex=True, inplace=True)
books['publishDate'] = books['publishDate'].str.extract('(\d{4})$').fillna('')
books['original_publication_year'] = books['original_publication_year'].fillna(books['publishDate'])
books['description'] = books['description'].fillna(books['title'])

In [45]:
#impute null in pages with median
books['pages'] = books['pages'].fillna(books['pages'].median())

In [46]:
#drop unnecessary columns with null
books = books.drop(['isbn', 'original_title', 'isbn13', 'publishDate'], axis=1)

In [47]:
#check missing value
books.isnull().values.any()

False

### Feature Engineering

In [48]:
# Keep important columns, drop the rest

#keep important columns, drop the rest
cols_to_keep = ['book_id', 'title', 'authors', 'original_publication_year', 'pages', 'description', 'genres', 'average_rating', 'ratings_count', 'books_count','small_image_url']
books = books[cols_to_keep]

In [49]:
#delete unnecessary characters from authors, genres and description column
col_trans = ['authors', 'genres', 'description']
for col in col_trans:
    books[col].replace(r"[\[\]\-\\\/\"—'()|:]", "", regex=True, inplace=True) 
books['description'].replace({"\n" : " ", "isbn13" : "", "isbn" : "", r"[0-9]{8,}" : "", r"[.,]" : ""}, regex=True, inplace=True)
books['description'] = books['description'].str.lower()

In [50]:
#rename column and change year, pages, and book_count to integer
books.rename(columns = {'original_publication_year':'year'}, inplace = True)
books[['year', 'pages', 'books_count']] = books[['year', 'pages', 'books_count']].astype(int)

In [51]:
books.sample(10)

,book_id,title,authors,year,pages,description,genres,average_rating,ratings_count,books_count,small_image_url
899,959,A Portrait of the Artist as a Young Man,"James Joyce, Seamus Deane",1916,329,the portrayal of stephen dedaluss dublin child...,"classics, fiction",3.60,92569,766,https://s.gr-assets.com/assets/nophoto/book/50...
9062,6922,Leading Change,John P. Kotter,1988,208,john kotter’s nowlegendary eightstep process f...,"business, nonfiction, selfhelp, psychology",4.01,12245,25,https://s.gr-assets.com/assets/nophoto/book/50...
954,1022,"Heir of Fire (Throne of Glass, #3)",Sarah J. Maas,2014,565,celaena has survived deadly contests and shatt...,"fantasy, youngadult, romance, fiction, paranormal",4.53,123843,46,https://images.gr-assets.com/books/1460846511s...
5200,5861,"Appointment with Death (Hercule Poirot, #19)",Agatha Christie,1938,303,among the towering red cliffs of petra like so...,"mystery, fiction, crime, classics, thriller",3.84,20793,210,https://images.gr-assets.com/books/1308808730s...
5383,6095,Essentialism: The Disciplined Pursuit of Less,Greg McKeown,2014,260,have you ever found yourself stretched too thi...,"nonfiction, selfhelp, business, psychology, ph...",3.99,16137,29,https://images.gr-assets.com/books/1403165375s...
3587,3904,"Anne of Ingleside (Anne of Green Gables, #6)",L.M. Montgomery,1939,274,anne is the mother of five with never a dull m...,"classics, fiction, youngadult, historicalficti...",4.05,36641,166,https://images.gr-assets.com/books/1285712495s...
5277,5962,"If You're Reading This, It's Too Late (Secret,...","Pseudonymous Bosch, Gilbert Ford",2008,385,beware!dangerous secrets lie between the pages...,"mystery, fantasy, fiction, youngadult",4.23,18310,34,https://images.gr-assets.com/books/1344268358s...
2742,2947,Dragons of Winter Night (Dragonlance: Chronicl...,"Margaret Weis, Tracy Hickman",1985,358,they won their first real battle in the war fo...,"fantasy, fiction, youngadult",4.13,31341,60,https://s.gr-assets.com/assets/nophoto/book/50...
5991,6877,A House for Mr Biswas,V.S. Naipaul,1961,623,mohun biswas has spent his 46 years of life st...,"fiction, classics, contemporary",3.82,13073,54,https://s.gr-assets.com/assets/nophoto/book/50...
5962,6839,Suicide Notes,Michael Thomas Ford,2008,295,im not crazy i dont see what the big deal is a...,"youngadult, contemporary, fiction, psychology",3.92,14749,10,https://images.gr-assets.com/books/1424990334s...


In [53]:
books.to_csv('books_cleaned.csv')